<a href="https://colab.research.google.com/github/luispuli/Final-Project/blob/master/Interactivo_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import cross_val_score
from nltk.stem import PorterStemmer
import gspread
import ipywidgets as widgets


# Librerías Google y credenciales Google
from google.colab import files
from google.colab import auth
from google.auth import default
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
wb = gc.open_by_key("1F1CO_3cDV1mz49VGI8CAhlV5FD34ae8_VCsoAkY8wmI")
sheet = wb.worksheet('CatálogoConsolidado')
data = sheet.get_all_values()
Sample = pd.DataFrame(data)
Sample.columns = Sample.loc[0,:]
Sample.drop(0,axis = 0, inplace = True)

In [4]:
wb = gc.open_by_key("19J2GOGRs17-Rk6Tup9FfUwJtsfpslg9Zf8Pe7jhhKg0")
sheet = wb.worksheet('Agrupadores Globales')
data = sheet.get_all_values()
Campos = pd.DataFrame(data)
Campos.columns = Campos.loc[0,:]
Campos.drop(0,axis = 0, inplace = True)

In [5]:
# Get relevant attributes and clean their names
Sample = Sample[["Agrupador GLOBAL N2","Servicio (Entregable)", "Descripción del servicio"]].rename(columns = {
                                                                             "Agrupador GLOBAL N2":"Group",
                                                                                                            "Servicio (Entregable)":"s",
                                                                             "Descripción del servicio":"de"})

In [6]:
Sample = Sample.assign(Description = Sample.de + " " + Sample.s).drop(["s","de"],axis = 1)

In [7]:
# Get relevant attributes and clean their names
Campos = Campos[["DESCRIPCIÓN","AGRUPADOR_GLOBAL_N2"]].rename(columns = {"DESCRIPCIÓN":"Description",
                                                                             "AGRUPADOR_GLOBAL_N2":"Group"})

In [8]:
Campos = Campos.assign(Description = Campos.Description.str.lower()).assign(Group_n = lambda x: x.Group.str.extract("(^\d+)"))

In [9]:
Campos = Campos.drop("Group_n",axis =1)
Sample = pd.concat([Campos,Sample])

In [10]:
Sample = Sample.loc[~Sample.Group.str.contains("99")]

In [11]:
Sample = Sample.assign(Description = Sample.Description.str.lower())

In [12]:
Sample = Sample.reset_index(names = "n").reset_index(names = "Doc").drop("n",axis = 1)

In [13]:
# Split into tokens and reshape the data to one row per token
Sample = Sample.assign(Description = Sample.Description.str.split()).explode("Description")
Sample = Sample.dropna()

In [14]:
# Import natural language library and extract the collection of Spanish stopwords
import nltk
nltk.download('stopwords')
Sample = Sample.reset_index().drop("index", axis = 1)
stopword_es = nltk.corpus.stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
# Remove punctuation
Sample = Sample.assign(Description = Sample.Description.str.replace('([&%—●•:–>.€ª,;()-/])',''))
Sample = Sample.assign(Description = Sample.Description.str.replace('"',''))
Sample['Description'] = Sample['Description'].str.replace(r'“|”', '', regex=True)
Sample = Sample.assign(Description = Sample.Description.str.replace('[',''))
Sample = Sample.assign(Description = Sample.Description.str.replace(']',''))
Sample = Sample.assign(Description = Sample.Description.str.replace('(\d+)',''))
Sample = Sample.reset_index()

<ipython-input-15-1f942775ef30>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  Sample = Sample.assign(Description = Sample.Description.str.replace('([&%—●•:–>.€ª,;()-/])',''))
<ipython-input-15-1f942775ef30>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Sample = Sample.assign(Description = Sample.Description.str.replace('[',''))
<ipython-input-15-1f942775ef30>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Sample = Sample.assign(Description = Sample.Description.str.replace(']',''))
<ipython-input-15-1f942775ef30>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  Sample = 

In [16]:
# Filter Stopwords
position = []
for i in range(Sample.shape[0]):
    if Sample.Description[i] in stopword_es:
        position.append(i)
Sample.drop(position, inplace= True)

In [17]:
from snowballstemmer import stemmer
stem = []
stemmer_es = stemmer("spanish")
for word in Sample.Description:
  stem.append(stemmer_es.stemWord(word))
Sample["Description"] = stem

In [18]:
Sample = Sample.groupby(["Doc","Group"]).Description.apply(" ".join).reset_index()

In [19]:
Sample = Sample.loc[~Sample.Group.str.contains("(\*)")]

<ipython-input-19-14448eee26cf>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Sample = Sample.loc[~Sample.Group.str.contains("(\*)")]


In [20]:
def check_presence(text):
    specific_attribute = 'riesg'  # Replace with your specific attribute

    # Check if the specific attribute is in the text
    if specific_attribute in text.lower():
        return 1  # Attribute is present
    else:
        return 0  # Attribute is not present

In [21]:
# Apply the function to create the new attribute
Sample['Atrr_1'] = Sample['Description'].apply(check_presence)

In [22]:
Sample['Description'] = Sample['Description'].str.replace(r'\s+', ' ', regex=True)

## Model

In [23]:
X = Sample.drop(columns=['Group'])  # Replace 'target_column' with the name of your target variable
y = Sample['Group']

test_size = 0.1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42,shuffle=True)




In [24]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_tfidf = tfidf_vectorizer.fit_transform(X_train["Description"])
#best_accuracy = 0
#best_estimator = 0
import scipy.sparse as sp

# Convert "Attr_1" to a sparse matrix
attr_1_sparse = sp.csr_matrix(X_train.Atrr_1.values.reshape(-1, 1))

# Concatenate the TF-IDF matrix and "Attr_1" matrix horizontally
X_combined = sp.hstack([X_tfidf, attr_1_sparse])
#for estimator in estimators:
  # Create the RandomForestClassifier with the current max_depth
classifier = RandomForestClassifier(n_estimators=1000, random_state=42, class_weight= "balanced_subsample")

    # Perform 5-fold cross-validation and calculate the mean accuracy
    #scores = cross_val_score(classifier, X_tfidf, y_train, cv=5, scoring='accuracy')
   # mean_accuracy = np.mean(scores)
classifier.fit(X_tfidf,y_train)
    # Check if this max_depth yields a better accuracy
    #if mean_accuracy > best_accuracy:
       # best_accuracy = mean_accuracy
      #  best_estimator = estimator


RandomForestClassifier(class_weight='balanced_subsample', n_estimators=1000,
                       random_state=42)

## Input text



In [95]:
text = input("Enter your service description:")

Text(value='', placeholder='Enter text')

## Operations

In [78]:
Sample = pd.DataFrame({"Description": [text]})

In [79]:
Sample = Sample.assign(Description = Sample.Description.str.lower())

In [80]:
Sample = Sample.reset_index(names = "n").reset_index(names = "Doc").drop("n",axis = 1)

In [81]:
# Split into tokens and reshape the data to one row per token
Sample = Sample.assign(Description = Sample.Description.str.split()).explode("Description")

In [82]:
# Import natural language library and extract the collection of Spanish stopwords
import nltk
nltk.download('stopwords')
Sample = Sample.reset_index().drop("index", axis = 1)
stopword_es = nltk.corpus.stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
Sample = Sample.assign(Description = Sample.Description.str.replace('([&%—●•:–>.€ª,;()-/])',''))
Sample = Sample.assign(Description = Sample.Description.str.replace('"',''))
Sample['Description'] = Sample['Description'].str.replace(r'“|”', '', regex=True)
Sample = Sample.assign(Description = Sample.Description.str.replace('[',''))
Sample = Sample.assign(Description = Sample.Description.str.replace(']',''))
Sample = Sample.assign(Description = Sample.Description.str.replace('(\d+)',''))
Sample = Sample.reset_index()

<ipython-input-83-2b4dc57221d3>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Sample = Sample.assign(Description = Sample.Description.str.replace('([&%—●•:–>.€ª,;()-/])',''))
<ipython-input-83-2b4dc57221d3>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Sample = Sample.assign(Description = Sample.Description.str.replace('[',''))
<ipython-input-83-2b4dc57221d3>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Sample = Sample.assign(Description = Sample.Description.str.replace(']',''))
<ipython-input-83-2b4dc57221d3>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  Sample = 

In [84]:
# Filter Stopwords
position = []
for i in range(Sample.shape[0]):
    if Sample.Description[i] in stopword_es:
        position.append(i)
Sample.drop(position, inplace= True)

In [85]:
from snowballstemmer import stemmer

# Create a stemmer object for Spanish
stemmer_es = stemmer("spanish")

# Define a function to perform stemming only on text data
def stem_text(text):
    if isinstance(text, str):  # Check if it's a string
        words = text.split()
        stemmed_words = [stemmer_es.stemWord(word) for word in words]
        return ' '.join(stemmed_words)
    return text  # Return the original value for non-text data

# Apply stemming to the "Description" column while preserving non-text values
Sample["Description"] = Sample["Description"].apply(stem_text)

In [86]:
Sample = Sample.assign(Description = Sample.Description.astype("str"))
Sample = Sample.groupby(["Doc"]).Description.apply(" ".join).reset_index()

In [87]:
# Vectorize the new text documents using the same TF-IDF vectorizer
new_text_tfidf = tfidf_vectorizer.transform(Sample["Description"])


# Make probability predictions for the new text documents
predicted_probabilities = classifier.predict_proba(new_text_tfidf)

# Get the top two predicted class labels for each text document
top_three_predictions = []
for probs in predicted_probabilities:
    # Sort the probabilities and get the indices of the top two
    top_three_indices = sorted(range(len(probs)), key=lambda i: probs[i], reverse=True)[:4]

    # Get the corresponding class labels for the top two indices
    top_three_classes = [classifier.classes_[i] for i in top_three_indices]

    top_three_predictions.append(top_three_classes)

# top_three_predictions now contains the top two predicted class labels for each text document

In [88]:
preds = pd.DataFrame(top_three_predictions).rename(columns = {0:"Pred_1",
                                                    1:"Pred_2",
                                                            2: "Pred_3",
                                                              3: "Pred_4"})

## Output

In [94]:
preds

,Pred_1,Pred_2,Pred_3,Pred_4
0,021- Información financiera regulatoria no púb...,049- Resto de reporting interno,110- Data,023- ILAAP (Internal Liquidity Adequacy Assess...
